In [10]:
import pandas as pd  
from sklearn.model_selection import GroupShuffleSplit

In [50]:
import os
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/MyDrive/Web Mining"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive',force_remount=True)  
os.chdir("/content/drive/MyDrive/Web Mining") 

Google Drive already mounted


In [51]:
cb = pd.read_csv('./Final Data/single_bert_test.csv')
print(cb)
cb.describe()

         UserID  MovieID    Rating  Ranking
0          3949       75  3.621008        1
1          3949     1463  3.613014        2
2          3949     2605  3.612508        3
3          3949     1830  3.611560        4
4          3949     3055  3.608603        5
...         ...      ...       ...      ...
4081463     926     1063  2.712190     3358
4081464     926     2575  2.671105     3359
4081465     926     1185  2.647367     3360
4081466     926     1296  2.642510     3361
4081467     926     1474  2.490881     3362

[4081468 rows x 4 columns]


,UserID,MovieID,Rating,Ranking
count,4.081468e+06,4.081468e+06,4.081468e+06,4.081468e+06
mean,3.105189e+03,1.680500e+03,3.353011e+00,1.681500e+03
std,1.804769e+03,9.705259e+02,4.060278e-01,9.705259e+02
min,2.000000e+00,0.000000e+00,1.281449e+00,1.000000e+00
25%,1.512000e+03,8.400000e+02,3.100069e+00,8.410000e+02
50%,3.135500e+03,1.680500e+03,3.380459e+00,1.681500e+03
75%,4.770000e+03,2.521000e+03,3.632778e+00,2.522000e+03
max,6.038000e+03,3.361000e+03,4.529549e+00,3.362000e+03


In [63]:
users = pd.read_csv("./Abhay/users_grouped.csv", skipinitialspace=True)
print(users.head())
users = users.drop(columns=['Gender', 'Age', 'Occupation'])
print(users)
gs = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=42)
train_ix, test_ix = next(gs.split(users, groups=users["Group"]))
# print(users.head())
# print(len(users))
train = users.loc[train_ix]
# print(train) 
test = users.loc[test_ix]

   UserID  Cluster Gender  Age  Occupation  Group
0    6040        0      M   25           6      1
1    3961        0      M   25           1      1
2    3966        0      M   25           0      1
3    3968        0      M   25           2      1
4    1835        0      M   25          19      1
      UserID  Cluster  Group
0       6040        0      1
1       3961        0      1
2       3966        0      1
3       3968        0      1
4       1835        0      1
...      ...      ...    ...
6035    3904       19    596
6036     940       19    596
6037    2217       19    596
6038    5063       19    596
6039       1       19    579

[6040 rows x 3 columns]


In [64]:
ratings = pd.read_csv("./ratings_new.csv")
ratings = ratings.drop('Timestamp', axis=1) 
print(ratings.head())

act_ratings_test = ratings.copy() 
act_ratings_test = act_ratings_test.loc[act_ratings_test['UserID'].isin(test['UserID'])]
act_ratings_test.reset_index(drop=True, inplace=True) # made sure the users are all in test! 

print(len(act_ratings_test))

   UserID  MovieID  Rating
0       1     1193       5
1       1      661       3
2       1      914       3
3       1     3408       4
4       1     2355       5
209646


In [65]:
act_ratings_test = act_ratings_test.drop(['Rating'], axis=1)
act_ratings_test

,UserID,MovieID
0,2,1357
1,2,3068
2,2,1537
3,2,647
4,2,2194
...,...,...
209641,6038,1387
209642,6038,2700
209643,6038,2716
209644,6038,3396


In [66]:
pred_ratings_test = pd.merge(cb, act_ratings_test, on=['UserID', 'MovieID'], how="right")

In [67]:
pred_ratings_test

,UserID,MovieID,Rating,Ranking
0,2,1357,3.465516,549.0
1,2,3068,3.318433,2689.0
2,2,1537,3.380338,2113.0
3,2,647,3.362192,2312.0
4,2,2194,3.441492,1034.0
...,...,...,...,...
209641,6038,1387,3.180861,3192.0
209642,6038,2700,3.485532,1356.0
209643,6038,2716,3.427263,2209.0
209644,6038,3396,NaN,NaN


In [68]:
len(pred_ratings_test)

209646

In [69]:
pred_ratings_test = pred_ratings_test.drop(['Ranking'], axis=1)

In [70]:
grouped = pd.merge(pred_ratings_test, users, on="UserID", how="left")

In [71]:
grouped.describe()

,UserID,MovieID,Rating,Cluster,Group
count,209646.000000,209646.000000,185434.000000,209646.000000,209646.000000
mean,3083.464101,1872.295880,3.222384,5.015808,263.740062
std,1788.643743,1096.757168,0.417052,5.528298,168.343882
min,2.000000,1.000000,1.392991,0.000000,3.000000
25%,1451.000000,1029.000000,2.972291,0.000000,111.000000
50%,3089.000000,1883.000000,3.247076,2.000000,239.000000
75%,4704.000000,2788.000000,3.514992,9.000000,389.000000
max,6038.000000,3952.000000,4.482568,19.000000,593.000000


In [72]:
grouped_ratings = pd.DataFrame(columns=['Group', 'MovieID', 'Rating'])
grouped_ratings

,Group,MovieID,Rating


In [73]:
def aggregation(method): 
  if (method == 'average'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].mean() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    # final['Rating'] = final.apply(lambda x: x['Rating'] / x['UserCount'], axis=1) 
    return final
  elif (method == 'least_misery'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].min() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final
  elif (method == 'most_pleasure'): 
    test = grouped.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].max() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final
  elif (method == 'average_without_misery'): 
    test = grouped.copy()
    test = test[test['Rating'] > 2.5]
    test = test.groupby(by=['Group', 'MovieID'], as_index=False)['Rating'].max() 
    test2 = users.groupby('Group')['UserID'].agg('count').reset_index()
    final = pd.merge(test, test2, on=["Group"], how="left")
    final = final.rename(columns = {'UserID':'UserCount'})
    return final

In [74]:
import os
def convert_df_to_csv(dataframe, method): 
  path='./Final Data'

  dataframe['Ranking'] = dataframe.groupby(by=['Group'], as_index=False)['Rating'].rank("dense", ascending=False)
  dataframe = pd.merge(grouped, dataframe, on=['Group', 'MovieID'], how='left')
  dataframe = dataframe.drop(['Rating_y', 'UserCount', 'Ranking', 'Cluster', 'Group'], axis=1)
  dataframe = dataframe.rename(columns = {'Rating_x':'Rating'})
  name = 'user_ratings_cb_' + method
  name = name + '.csv' 
  output_file = os.path.join(path, name)
  dataframe.to_csv(output_file, index=False)

In [75]:
ave_df = aggregation('average')
convert_df_to_csv(ave_df, 'average')

In [76]:
test = pd.read_csv('./Final Data/user_ratings_cf_average.csv', index_col=False)
print(test)

        UserID  MovieID    Rating
0            2     1357  3.650502
1            2     3068  3.589013
2            2     1537  3.635573
3            2      647  3.565670
4            2     2194  3.860406
...        ...      ...       ...
209641    6038     1387  3.796086
209642    6038     2700  3.666036
209643    6038     2716  3.774065
209644    6038     3396  3.645155
209645    6038     1079  3.736976

[209646 rows x 3 columns]


In [77]:
least_misery_df = aggregation('least_misery')
convert_df_to_csv(least_misery_df, 'least_misery')

In [78]:
most_pleasure_df = aggregation('most_pleasure')
convert_df_to_csv(most_pleasure_df, 'most_pleasure')

In [79]:
average_without_misery_df = aggregation('average_without_misery')
convert_df_to_csv(average_without_misery_df, 'average_without_misery')

In [80]:
test = pd.read_csv('./Final Data/user_ratings_cf_average_without_misery.csv', index_col=False)
print(test)

        UserID  MovieID    Rating
0            2     1357  3.650502
1            2     3068  3.589013
2            2     1537  3.635573
3            2      647  3.565670
4            2     2194  3.860406
...        ...      ...       ...
209641    6038     1387  3.796086
209642    6038     2700  3.666036
209643    6038     2716  3.774065
209644    6038     3396  3.645155
209645    6038     1079  3.736976

[209646 rows x 3 columns]
